# Libraries


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
pip install -U deepctr-torch

In [16]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import json
import time
from datetime import datetime, timedelta
from deepctr_torch.layers.interaction import FM

from sklearn.metrics import log_loss,roc_auc_score,confusion_matrix, accuracy_score, precision_recall_curve, f1_score, precision_score, recall_score
import json
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('using gpu!')
else:
  device = torch.device('cpu')
  print('using cpu!')

using gpu!


In [17]:
dense_features = ['I_'+str(i) for i in range(13)]
sparse_features = ['C_'+str(i) for i in range(26)]
df_columns = ['click'] + dense_features + sparse_features
df_columns

['click',
 'I_0',
 'I_1',
 'I_2',
 'I_3',
 'I_4',
 'I_5',
 'I_6',
 'I_7',
 'I_8',
 'I_9',
 'I_10',
 'I_11',
 'I_12',
 'C_0',
 'C_1',
 'C_2',
 'C_3',
 'C_4',
 'C_5',
 'C_6',
 'C_7',
 'C_8',
 'C_9',
 'C_10',
 'C_11',
 'C_12',
 'C_13',
 'C_14',
 'C_15',
 'C_16',
 'C_17',
 'C_18',
 'C_19',
 'C_20',
 'C_21',
 'C_22',
 'C_23',
 'C_24',
 'C_25']

# Load Data

In [18]:
total_len = 45840617
train_len = 40000000
test_len = 5840617

In [19]:
dense_feat_len = 0
sparse_feat_len = 0
nrow = 20000

df_trainX_sparse1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train1.csv',nrows=nrow)
sparse_feat_len += df_trainX_sparse1.shape[1]

df_trainX_sparse2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train2.csv',nrows=nrow)
sparse_feat_len += df_trainX_sparse2.shape[1]

df_trainX_sparse3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train3.csv',nrows=nrow)
sparse_feat_len += df_trainX_sparse3.shape[1]

df_trainX_dense = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/dense_train.csv',nrows=nrow)
dense_feat_len += df_trainX_dense.shape[1]

df_y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/train_label.csv',nrows=nrow)

df_train = df_y.join([df_trainX_dense,df_trainX_sparse1,df_trainX_sparse2,df_trainX_sparse3])
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 40 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   click   20000 non-null  int64  
 1   I_0     20000 non-null  float64
 2   I_1     20000 non-null  float64
 3   I_2     20000 non-null  float64
 4   I_3     20000 non-null  float64
 5   I_4     20000 non-null  float64
 6   I_5     20000 non-null  float64
 7   I_6     20000 non-null  float64
 8   I_7     20000 non-null  float64
 9   I_8     20000 non-null  float64
 10  I_9     20000 non-null  float64
 11  I_10    20000 non-null  float64
 12  I_11    20000 non-null  float64
 13  I_12    20000 non-null  float64
 14  C_0     20000 non-null  int64  
 15  C_1     20000 non-null  int64  
 16  C_2     20000 non-null  int64  
 17  C_3     20000 non-null  int64  
 18  C_4     20000 non-null  int64  
 19  C_5     20000 non-null  int64  
 20  C_6     20000 non-null  int64  
 21  C_7     20000 non-null  int64  
 22

In [20]:
df_train.head()

,click,I_0,I_1,I_2,I_3,I_4,I_5,I_6,I_7,I_8,...,C_16,C_17,C_18,C_19,C_20,C_21,C_22,C_23,C_24,C_25
0,0,0.035714,0.001783,0.026455,0.000000,0.004032,0.003571,0.071429,0.041667,0.186598,...,3854202482,4114618041,568184265,2972002973,129309004,0,974593739,3318023300,3904386055,2535972118
1,0,0.071429,0.001337,0.232804,0.026316,0.000298,0.007143,0.009524,0.041667,0.004124,...,130367684,2957919856,568184265,1480633834,1626743326,0,974593739,1139883659,3904386055,1931228757
2,0,0.071429,0.001337,0.005291,0.368421,0.002238,0.079464,0.019048,0.041667,0.252577,...,2399067775,873599373,0,0,3850880102,2905629419,974593739,991444060,0,0
3,0,0.000000,0.399465,0.000000,0.000000,0.012815,0.000000,0.000000,0.000000,0.000000,...,512280399,1961833730,0,0,1798986918,0,974593739,2434245450,0,0
4,0,0.107143,0.000892,0.000000,0.000000,0.000006,0.000000,0.014286,0.000000,0.000000,...,512280399,649316263,0,0,566841706,0,851920782,3008311592,0,0


In [21]:
class MyDataset(Dataset):
    def __init__(self, df,):
        self.df = df
        self.X = df.drop(columns='click').values
        self.y = df['click'].values
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        data_x = self.X[idx]
        data_y = self.y[idx]


        return data_x, data_y


train_data = MyDataset(df_train[:-10000])
valid_data = MyDataset(df_train[-10000:])
#test_data = MyDataset(test_df)
batch_size = 128
num_workers = 2
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
#test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

iter_x, iter_y = next(iter(train_loader))
#print(iter_y)
#print(iter_x.shape)
iter_x

tensor([[3.5714e-02, 1.7833e-03, 2.6455e-02,  ..., 3.3180e+09, 3.9044e+09,
         2.5360e+09],
        [7.1429e-02, 1.3375e-03, 2.3280e-01,  ..., 1.1399e+09, 3.9044e+09,
         1.9312e+09],
        [7.1429e-02, 1.3375e-03, 5.2910e-03,  ..., 9.9144e+08, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 3.1654e-02, 0.0000e+00,  ..., 0.0000e+00, 2.0454e+06,
         1.4553e+09],
        [0.0000e+00, 6.2416e-03, 1.8519e-01,  ..., 2.8108e+09, 2.0454e+06,
         3.3804e+09],
        [0.0000e+00, 1.3375e-03, 0.0000e+00,  ..., 2.4981e+09, 7.3758e+08,
         3.1098e+09]], dtype=torch.float64)

# Sparse Field Preprocessing

In [ ]:

file_path = '/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/feature.json'

# Open the JSON file and load its contents into a Python dictionary
with open(file_path, 'r') as file:
    features_dict = json.load(file)

features_dict = json.loads(features_dict)
features_dict['C_0']

In [ ]:

large_feat = []
for feat in features_dict:
    if len(features_dict[feat]) > 100000:
      large_feat.append(feat)
large_feat

In [ ]:
df_temp = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train1.csv',usecols=['C_2','C_3'])
df_values = df_temp['C_2'].value_counts().reset_index()
df_values = df_values[df_values['C_2'] > 100]
features_dict['C_2'] = list(df_values['index'].values)

df_values = df_temp['C_3'].value_counts().reset_index()
df_values = df_values[df_values['C_3'] > 100]
features_dict['C_3'] = list(df_values['index'].values)

In [ ]:
df_temp = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train2.csv',usecols=['C_11'])
df_values = df_temp['C_11'].value_counts().reset_index()
df_values = df_values[df_values['C_11'] > 100]
features_dict['C_11'] = list(df_values['index'].values)



In [ ]:
df_temp = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train3.csv',usecols=['C_15','C_20','C_23','C_25'])
df_values = df_temp['C_15'].value_counts().reset_index()
df_values = df_values[df_values['C_15'] > 100]
features_dict['C_15'] = list(df_values['index'].values)

df_values = df_temp['C_20'].value_counts().reset_index()
df_values = df_values[df_values['C_20'] > 100]
features_dict['C_20'] = list(df_values['index'].values)

df_values = df_temp['C_23'].value_counts().reset_index()
df_values = df_values[df_values['C_23'] > 100]
features_dict['C_23'] = list(df_values['index'].values)

df_values = df_temp['C_25'].value_counts().reset_index()
df_values = df_values[df_values['C_25'] > 100]
features_dict['C_25'] = list(df_values['index'].values)

In [ ]:
def custom_serializer(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

# Convert the dictionary to a JSON string using the custom serializer
r = json.dumps(features_dict, default=custom_serializer)
filename = "/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/updated_feature.json"

# Open the file in write mode ('w') and use json.dump() to write the JSON data
with open(filename, 'w') as file:
    json.dump(r, file, indent=4)

# Field indexing

In [22]:
file_path = '/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/updated_feature.json'

# Open the JSON file and load its contents into a Python dictionary
with open(file_path, 'r') as file:
    features_dict = json.load(file)

features_dict = json.loads(features_dict)
features_dict['C_0']

[1761418852,
 679372879,
 2364568165,
 98275684,
 1134183588,
 605374176,
 3193477969,
 1520359856,
 1016956805,
 342162125,
 2119970804,
 2592715628,
 4212608619,
 1543146165,
 2270503831,
 2927815201,
 4101224668,
 442471022,
 1850357391,
 3025520343,
 402035541,
 1755471301,
 164236161,
 2525297997,
 4278209251,
 1974218726,
 1589391762,
 967779847,
 1170965705,
 916829183,
 2825881377,
 1246660036,
 3306338393,
 1106095165,
 3474921515,
 4097112769,
 4037227861,
 2315465859,
 2355864583,
 996529735,
 619553622,
 1705695884,
 2596572863,
 1135988299,
 2522921339,
 1822666566,
 2162480981,
 1444674004,
 3652907944,
 3568340312,
 4224015790,
 3689298987,
 1692891879,
 731037906,
 3125101410,
 3175836948,
 760127787,
 3277703553,
 2245823812,
 3670613032,
 763316445,
 3503711093,
 3943323301,
 1582550072,
 3298313251,
 689666978,
 242793798,
 1233154168,
 1430970331,
 2552461107,
 169271764,
 2706174266,
 3662610191,
 3442906542,
 3543180912,
 3585186967,
 2450904002,
 3340428177,
 379

In [23]:
num_emb = 0
max_field_len = 0
start_list = [0]
tensor_list = []
index_list = []
for feature in features_dict:
    start_list.append(len(features_dict[feature])+2+start_list[-1])
    num_emb += (len(features_dict[feature])+2)
    if len(features_dict[feature]) > max_field_len:
        max_field_len = len(features_dict[feature])
start_list

[0,
 1462,
 2047,
 22903,
 46543,
 46850,
 46876,
 59395,
 60030,
 60035,
 153182,
 158867,
 180161,
 183357,
 183386,
 198380,
 220747,
 220759,
 226413,
 228588,
 228594,
 250357,
 250377,
 250394,
 265425,
 265532,
 277753]

In [24]:
tensor_list = []
index_list = []
for feature in features_dict:
    tensor_list.append(F.pad(torch.tensor(features_dict[feature],dtype=torch.int64), pad=(0, max_field_len - len(features_dict[feature]))))
    index_list.append(F.pad(torch.tensor(list(range(1,1+len(features_dict[feature]))),dtype=torch.int64), pad=(0, max_field_len - len(features_dict[feature]))))


tensor_matrix = torch.stack(tensor_list)
index_matrix = torch.stack(index_list)
tensor_matrix

tensor([[1761418852,  679372879, 2364568165,  ...,          0,          0,
                  0],
        [2162322587, 4040097828,  173120604,  ...,          0,          0,
                  0],
        [         0, 3492987491,   47159414,  ...,          0,          0,
                  0],
        ...,
        [1071331371, 3008311592,  991444060,  ...,          0,          0,
                  0],
        [3904386055,          0, 3935970412,  ...,          0,          0,
                  0],
        [         0, 1238795398, 3360443116,  ...,          0,          0,
                  0]])

# DeepFM Architecture

In [25]:
class DeepFM(nn.Module):
    def __init__(self,drop_rate=0.9,dense_len=dense_feat_len,sparse_len=sparse_feat_len,emb_dim=5,MAX_DIM=num_emb,hidden_size=200,start_points=start_list[:-1],tensor_matrix=tensor_matrix,target_matrix=index_matrix):
        super(DeepFM, self).__init__()

        self.hidden_size = hidden_size
        self.num_dense_features = dense_feat_len
        self.num_sparse_features = sparse_feat_len
        self.emb_dim = emb_dim
        self.MAX_DIM = MAX_DIM
        self.start_points = torch.tensor(start_points).to(device)
        self.tensor_matrix = tensor_matrix.to(device)
        self.target_matrix = target_matrix.to(device)
        self.embeddings = nn.Embedding(MAX_DIM,emb_dim)
        self.intxlayer = FM()
        self.linear_layer1 = nn.Linear(self.emb_dim*self.num_sparse_features+self.num_dense_features,self.hidden_size)
        self.linear_layer2 = nn.Linear(self.hidden_size,self.hidden_size)
        self.linear_layer3 = nn.Linear(self.hidden_size,self.hidden_size)
        self.final_layer = nn.Linear(self.hidden_size,1)
        self.drop_out = nn.Dropout(drop_rate)


    def forward(self,x):
        size = x.shape[0]
    #feature transformation preprocessing

        x_dense = x[:,:self.num_dense_features]
        x_sparse = x[:,self.num_dense_features:].long()

        x_sparse = x_sparse.reshape(size,-1,1)
        x_sparse = torch.eq(self.tensor_matrix,x_sparse)
        x_sparse, _ = torch.max(x_sparse * self.target_matrix,dim=2)
        x_sparse = x_sparse + self.start_points
        # embedding layer
        embs = self.embeddings(x_sparse)
        # Deep component
        x1 = torch.concat([x_dense,embs.reshape(size,-1)],dim=1)
        x1 = x1.reshape(size,-1).float()
        # MLP input
        x1 = F.relu(self.linear_layer1(x1))
        x1 = self.drop_out(x1)
        x1 = F.relu(self.linear_layer2(x1))
        x1 = self.drop_out(x1)
        x1 = F.relu(self.linear_layer3(x1))
        x1 = self.drop_out(x1)
        x1 = self.final_layer(x1)
        # feature interaction component
        x2 = self.intxlayer(embs)


        #sigmoid layer
        output = torch.flatten(F.sigmoid(x1+x2))

      #  output1 = F.log_softmax(logits,dim=1)

        return output

# Model Train

In [33]:
def fit(model, train_data,valid_data,optimizer,weight_positive=0.3,threshold=0.5):
        #train and output the loss of each epoch.
        max_loss = 30
        gt_labels = []
        pred_labels = []
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([weight_positive]).to(device))
        print('start training')

        for t, (batch_x, batch_y) in enumerate(train_data):
            optimizer.zero_grad()
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            total = model.forward(batch_x)
            pred = torch.sigmoid(total)
            loss = criterion(total, batch_y.float())
            loss.backward()
            optimizer.step()

            if(t%2000==0):

                print(t,'/',len(train_data))


            gt_label = batch_y.cpu().data.numpy()
            pred_proba = pred.cpu().data.numpy()
            gt_labels.append(gt_label)
            pred_labels.append(pred_proba)

        gt_labels, pred_labels = np.concatenate(gt_labels), np.concatenate(pred_labels)
        pred_labels = pred_labels.reshape(len(pred_labels),)
        prediction = pred_labels>threshold
        prediction = prediction.astype(int)
        lloss = log_loss(gt_labels,prediction)
        f1 = f1_score(gt_labels,prediction)
        print('training results, logloss:',lloss,'f1:',f1)



        all_loss = 0
        gt_labels = []
        pred_labels = []

        with torch.no_grad():
            for t, (batch_x, batch_y) in enumerate(valid_data):
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                pred = model.forward(batch_x)
                pred = torch.sigmoid(pred)
                gt_label = batch_y.cpu().data.numpy()
                pred_proba = pred.cpu().data.numpy()
                gt_labels.append(gt_label)
                pred_labels.append(pred_proba)
                loss = criterion(pred, batch_y.float())
                all_loss += loss.item()

        gt_labels, pred_labels = np.concatenate(gt_labels), np.concatenate(pred_labels)
        pred_labels = pred_labels.reshape(len(pred_labels),)
        prediction = pred_labels>threshold
        prediction = prediction.astype(int)
        lloss = log_loss(gt_labels,prediction)
        f1 = f1_score(gt_labels,prediction)

        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/DeepFFM_weights')

        print('valid results, logloss:',lloss,'f1:',f1,'BCEloss:',all_loss/t)

In [32]:


chunk_size = 40000000

df_trainX_sparse1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train1.csv',nrows=chunk_size)


df_trainX_sparse2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train2.csv',nrows=chunk_size)


df_trainX_sparse3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train3.csv',nrows=chunk_size)


df_trainX_dense = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/dense_train.csv',nrows=chunk_size)


df_y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/train_label.csv',nrows=chunk_size)
total_rows_to_read = 45840617 #45M
df_train = df_y.join([df_trainX_dense,df_trainX_sparse1,df_trainX_sparse2,df_trainX_sparse3])


train_data = MyDataset(df_train[:-1000000])
valid_data = MyDataset(df_train[-1000000:])
  #test_data = MyDataset(test_df)
batch_size = 256
num_workers = 2
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
lr = 1e-3
deepfm = DeepFM()
deepfm = deepfm.to(device)
optimizer = optim.Adam(deepfm.parameters(), lr=lr, weight_decay=5e-4)
for epoch in range(5):
    start_time = time.time()
    fit(deepfm,train_loader,valid_loader,optimizer,weight_positive=1)
    end_time = time.time()
    used_time = (end_time - start_time)/3600
    print('Epoch %d, used_hour=%.4f' % (epoch, used_time))




start training
0 / 152344
2000 / 152344
4000 / 152344
6000 / 152344
8000 / 152344
10000 / 152344
12000 / 152344
14000 / 152344
16000 / 152344
18000 / 152344
20000 / 152344
22000 / 152344
24000 / 152344
26000 / 152344
28000 / 152344
30000 / 152344
32000 / 152344
34000 / 152344
36000 / 152344
38000 / 152344
40000 / 152344
42000 / 152344
44000 / 152344
46000 / 152344
48000 / 152344
50000 / 152344
52000 / 152344
54000 / 152344
56000 / 152344
58000 / 152344
60000 / 152344
62000 / 152344
64000 / 152344
66000 / 152344
68000 / 152344
70000 / 152344
72000 / 152344
74000 / 152344
76000 / 152344
78000 / 152344
80000 / 152344
82000 / 152344
84000 / 152344
86000 / 152344
88000 / 152344
90000 / 152344
92000 / 152344
94000 / 152344
96000 / 152344
98000 / 152344
100000 / 152344
102000 / 152344
104000 / 152344
106000 / 152344
108000 / 152344
110000 / 152344
112000 / 152344
114000 / 152344
116000 / 152344
118000 / 152344
120000 / 152344
122000 / 152344
124000 / 152344
126000 / 152344
128000 / 152344
130

KeyboardInterrupt: 

In [ ]:
lr = 2e-4
deepfm = DeepFM()
deepfm.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/DeepFFM_weights'))
deepfm = deepfm.to(device)
optimizer = optim.Adam(deepfm.parameters(), lr=lr, weight_decay=5e-4)
for epoch in range(5):
    start_time = time.time()
    fit(deepfm,train_loader,valid_loader,optimizer,weight_positive=1)
    end_time = time.time()
    used_time = (end_time - start_time)/3600
    print('Epoch %d, used_hour=%.4f' % (epoch, used_time))

start training
0 / 152344
2000 / 152344
4000 / 152344
6000 / 152344
8000 / 152344
10000 / 152344
12000 / 152344
14000 / 152344
16000 / 152344


# End

In [ ]:
## load test

df_testX_sparse1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train1.csv',skiprows=(total_len-test_len),names=sparse_features[:7])


df_testX_sparse2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train2.csv',skiprows=(total_len-test_len),names=sparse_features[7:14])


df_testX_sparse3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/sparse_train3.csv',skiprows=(total_len-test_len),names=sparse_features[14:])


df_testX_dense = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/dense_train.csv',skiprows=(total_len-test_len),names=dense_features)


df_y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NN study/DeepFFM/Criteo Data/train_label.csv',skiprows=(total_len-test_len),names=['click'])

df_test = df_y.join([df_testX_dense,df_testX_sparse1,df_testX_sparse2,df_testX_sparse3])
df_test.info()